# Pernambuco State Wages in 2022
This is an initial analysis of 2022 wages data of government employees. These data are stored in https://dados.pe.gov.br/dataset/remuneracao-de-servidores.

In [1]:
import requests
import pandas as pd
import matplotlib.pyplot as plt

## Initial Analysis
First, we need to understand the data structure and content. The sample I'm using to check data is the December active employees file. The file is a JSON with only one key named "campos". The data of this key is a list of dictionaries.

In [2]:
def get_dataset(url: str) -> pd.DataFrame:
    response = requests.get(url)
    if response.status_code == 200:
        if url.endswith('.json'):
            data = response.json()['campos']
        return pd.DataFrame(data)
    else:
        print('Request to provided URL failed!')
    return None

In [3]:
sample = r'https://dados.pe.gov.br/dataset/7bbfeed7-3019-4c6a-bee7-e1b7411f616b/resource/68cb38cf-bae8-49d8-a30b-6cf48338ada7/download/2022_12_remuneracao_ativos.json'
df = get_dataset(sample)

Let's see the columns of this JSON:

In [4]:
df.columns

Index(['r_total_vantagens', 'r_imposto_renda', 'r_outras_vantagens',
       'r_vencimento_cargo', 'r_categoria', 'r_gratificacao_funcao', 'r_cargo',
       'r_desconto_excedente', 'r_descontos_compulsorios', 'r_matricula',
       'r_natalina', 'r_nome', 'r_outros_creditos', 'r_instituicao',
       'r_funcao', 'r_descontos_previdencia', 'r_valor_liquido',
       'r_remuneracao', 'r_ferias', 'r_cpf', 'r_descontos_faltas'],
      dtype='object')

By the metadata information from the datasource, we have:

* r_instituicao: Unit responsible for performing government functions.
* r_cpf: Masked individual taxpayer registration number.
* r_matricula: Numerical identification of the employee in a given agency or entity.
* r_nome: Full name of the employee.
* r_categoria: Indicates the type of bond of the employee with the agency or entity.
* r_cargo: Name of the position or job that the employee holds in the agency or entity.
* r_vencimento_cargo: Fixed portion of the remuneration without other advantages.
* r_funcao: Temporary designation for the performance of specific duties.
* r_gratificacao_funcao: Amount related to the gratified function or the commissioned position held by the employee.
* r_remuneracao: Amount corresponding to the sum of the fixed and variable portions of the position.
* r_ferias: Value of the 1/3 vacation bonus calculated based on the employee's remuneration.
* r_natalina: Amount corresponding to 1/12 of the remuneration that the employee is entitled to.
* r_outras_vantagens: Amount corresponding to indemnity advantages.
* r_total_vantagens: Total advantages received by the employee without deductions.
* r_desconto_excedente: Deduction of the excess remuneration.
* r_descontos_faltas: Deductions for absences and/or reimbursements to the State.
* r_descontos_previdencia: Amount deducted from the employee for the own pension scheme.
* r_imposto_renda: Amount corresponding to the income tax deduction.
* r_descontos_compulsorios: Compulsory deductions.
* r_valor_liquido: Total amount received by the employee after all deductions.
* r_outros_creditos: Amounts paid through financial execution, some of which are indemnity amounts received by the employee.

Let's see some statistical information about the numeric fields:

In [10]:
df.describe()

,r_total_vantagens,r_imposto_renda,r_outras_vantagens,r_vencimento_cargo,r_gratificacao_funcao,r_desconto_excedente,r_descontos_compulsorios,r_natalina,r_descontos_previdencia,r_valor_liquido,r_remuneracao,r_ferias,r_descontos_faltas
count,240592.000000,240592.000000,240592.000000,240592.000000,240592.000000,240592.000000,240592.000000,240592.000000,240592.000000,240592.000000,240592.000000,240592.000000,240592.000000
mean,3490.403097,406.234045,332.358127,2159.697203,223.150437,23.134294,762.640515,15.310101,323.686571,2727.769764,2911.786640,237.248878,9.585605
std,5243.506451,928.670259,915.535300,3214.452368,1137.287108,414.579118,1718.760318,140.510748,549.304183,3701.743455,4471.053722,747.310471,207.662772
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1584.400000,18.855000,0.000000,813.560000,0.000000,0.000000,138.420000,0.000000,87.215000,1425.230000,1320.000000,0.000000,0.000000
75%,5584.400000,406.930000,340.400000,3900.000000,0.000000,0.000000,971.100000,0.000000,546.000000,4613.300000,4316.210000,0.000000,0.000000
max,129461.690000,13478.510000,115033.750000,30365.430000,26971.380000,13389.030000,80261.960000,10925.060000,16057.910000,49199.730000,59004.230000,37586.850000,54780.850000


In [11]:
df['r_matricula'].value_counts()

r_matricula
3026        6
1490        6
4901        6
1716        6
3603        5
           ..
12770067    1
12895803    1
13359174    1
12247917    1
4532040     1
Name: count, Length: 218287, dtype: int64

The amount of rows in December 2022 is 240,592. Checking if there are more than one row for each "r_matricula", we notice that there are duplicated rows. This will be a future concern.

### Basic statistics of wages:
- Average: BRL 2,911.79
- Std: BRL 4,471.05
- Median: BRL 1,320.00
- Max: BRL 59,004.23

### Most common positions and average wages:
- Professor (Teacher): BRL 6,373.88
- Terceiro Sargento (Sergeant): BRL 5,006.30
- Médico (Doctor): BRL 3,831.82
- Cabo (Corporal): BRL 2,421.02
- Analista em Saúde (Health analyst): BRL 1,248.63
- Auxiliar em Saúde (Health assistant): BRL 1,071.07
- Técnico em Enfermagem-CTD (Nursing technician): BRL 915.58
- Soldado (Soldier): BRL 703.39

Let's see some sampling of data:

In [6]:
df.head()

,r_total_vantagens,r_imposto_renda,r_outras_vantagens,r_vencimento_cargo,r_categoria,r_gratificacao_funcao,r_cargo,r_desconto_excedente,r_descontos_compulsorios,r_matricula,...,r_nome,r_outros_creditos,r_instituicao,r_funcao,r_descontos_previdencia,r_valor_liquido,r_remuneracao,r_ferias,r_cpf,r_descontos_faltas
0,10344.28,1341.57,334.4,5004.94,SERVIDOR ESTATUTARIO EFETIVO,0.0,POLICIAL PENAL DO ESTADO,0.0,2742.95,2089572,...,VANDI ALVES DE LIMA,None,SERES - SECRETARIA EXECUTIVA DE RESSOCIALIZAÇÃO,NA,1401.38,7601.33,10009.88,0.0,***.877.934-**,0.00
1,7800.36,893.48,334.4,3682.98,SERVIDOR ESTATUTARIO EFETIVO,0.0,POLICIAL PENAL DO ESTADO,0.0,1949.12,3373223,...,FABRICIA MONTEIRO DUQUE,None,SERES - SECRETARIA EXECUTIVA DE RESSOCIALIZAÇÃO,NA,1031.23,5851.24,7465.96,0.0,***.913.194-**,24.41
2,11265.44,1663.69,334.4,5465.52,SERVIDOR ESTATUTARIO EFETIVO,0.0,POLICIAL PENAL DO ESTADO,0.0,3194.04,2089351,...,EVANDRO ALVES DOS SANTOS,None,SERES - SECRETARIA EXECUTIVA DE RESSOCIALIZAÇÃO,NA,1530.35,8071.40,10931.04,0.0,***.924.034-**,0.00
3,12083.48,1908.50,580.8,5004.94,SERVIDOR ESTATUTARIO EFETIVO,1392.8,POLICIAL PENAL DO ESTADO,0.0,3309.88,3374432,...,LILIANE DE CASSIA SILVA DE ARRUDA,None,SERES - SECRETARIA EXECUTIVA DE RESSOCIALIZAÇÃO,SUBSTITUICAO SIMBOLO-FGS-1,1401.38,8773.60,11502.68,0.0,***.007.124-**,0.00
4,11257.18,1576.99,580.8,5004.94,SERVIDOR ESTATUTARIO EFETIVO,566.5,POLICIAL PENAL DO ESTADO,0.0,2978.37,2156113,...,CASSEUS CLAY LEITE SILVA,None,SERES - SECRETARIA EXECUTIVA DE RESSOCIALIZAÇÃO,FUN GRAT SUPERVISAO 3 - FGS-3,1401.38,8278.81,10676.38,0.0,***.018.904-**,0.00


And the detected data types:

In [12]:
df.dtypes

r_total_vantagens           float64
r_imposto_renda             float64
r_outras_vantagens          float64
r_vencimento_cargo          float64
r_categoria                  object
r_gratificacao_funcao       float64
r_cargo                      object
r_desconto_excedente        float64
r_descontos_compulsorios    float64
r_matricula                  object
r_natalina                  float64
r_nome                       object
r_outros_creditos            object
r_instituicao                object
r_funcao                     object
r_descontos_previdencia     float64
r_valor_liquido             float64
r_remuneracao               float64
r_ferias                    float64
r_cpf                        object
r_descontos_faltas          float64
dtype: object